# Juliaで非線型方程式の解の精度保証付き数値計算

非線形方程式

$$
f(x) = 0,\quad f:\mathbb{R}^n \rightarrow \mathbb{R}^n
$$

の解を精度保証することを考える。


今回は、区間解析の標準的な手法となっているR. Krawczykによる解の検証手法、並びに区間ニュートン法による解の検証方法を紹介する。

## Krawczyk法

Krawczykの主張は、

**定理**　$X\in \mathbb{R}^n$を区間ベクトル（候補者集合）、$c = \mathrm{mid}(X)$、$R\simeq Df(c)^{-1}=J(c)^{-1}$、$E$ を単位行列とし、

\begin{equation}
K(X) = c-Rf(c)+(E-RDf(X))(X-c)
\end{equation}

としたとき、$K(X)\subset \mathrm{int}(X)$ ならば $X$ に $f(x)=0$ の解が唯一存在する。

というものである。（$\mathrm{int}(X)$：$X$ の内部）

この主張を成り立たせるためには、$g(x)=x-Rf(x)$ に対して、平均値形式と縮小写像の原理を適用する必要がある。

### Krawczyk写像

$X\in \mathbb{IR}^n$ を区間ベクトルとし、写像 $f$ に対する簡易ニュートン写像 $g:\mathbb{R}^n\rightarrow \mathbb{R}^n$ を次で定義する。

$$
g(x)=x-Rf(x)
$$

$R\in \mathbb{R}^{n\times n}$ は、$n$ 次元正方行列として、$c\in X$ におけるヤコビ行列 $J(c)$ の逆近似行列 $(R\simeq f'(c)^{-1})$ とする。

このとき、$R$ が正則であるならば、

$$
f(x)=0\Longleftrightarrow g(x)=x
$$

が成り立つ。

そして、写像 $g$ が、区間ベクトル $X$ から $X$ への縮小写像となれば、縮小写像の原理から、$f(x)=0$ の真の解 $\tilde{x}$ が $X$ 内に一意存在することが示せる。しかし、写像 $g$ の区間 $X$ における区間拡張 $s_{[]}(X)$ を考えると、常に $s_{[]}(X)=X-Rf_{[]}(X)\not\subset X$ となり、縮小写像の原理を示すことができない。そこで、区間演算における区間の増大を抑制するための基本手法である平均値形式を導入する。

$$
s(X):=\{s(x)\mid x\in X\} (\mbox{写像 $s$ の値域})
$$

$$
s_{[]}(X)\supset s(X)
$$

以下に、具体的な例を示す。

$$
F(x,y) = \left(\begin{array}{c}f(x,y)\\ g(x,y)\end{array}\right) = \left(\begin{array}{c}x^2 + y^2 - 1\\ x - y\end{array}\right)
$$

とし、候補区間 $X$ を $ [0.707107 ± 1.0\times 10^{-8}, 0.707107 ± 1.0\times 10^{-8}]$ とする。

区間拡張 $s_{[]}(X)$ を計算すると、

$$
s_{[]}(X)=X-Rf_{[]}(X) = [0.707106, 0.707107], [0.707106, 0.707107] \not\subset X
$$

となり、候補区間 $X$ に含まれないことがわかる。このような例を解決するために平均値形式を導入する。

In [51]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff
X = [0.707107 ± 1e-8, 0.707107 ± 1e-8];
c = mid.(X);
f(x, y) = x^2 + y^2 - 1;
g(x, y) = x - y;
F( (x, y) ) = [f(x, y); g(x, y)];
DF = ForwardDiff.jacobian(F,c);
R = inv(DF);
s = X - R*F(X);
@show s
@show s .⊂ X

s = Interval{Float64}[[0.707106, 0.707107], [0.707106, 0.707107]]
s .⊂ X = Bool[0, 0]


2-element BitArray{1}:
 0
 0

## 縮小写像の原理

**定義**　$X\subseteq \mathbb{R}^n$ として、写像 $T:X\rightarrow X$ を考える。このとき、全ての $x,y\in X$ において、距離 $d(x,y)$ を $d(x,y):=\|x-y\|_E$ で定義し、

$$
d(T(x),T(y))\le \alpha d(x,y)
$$

が成り立つ $\alpha\in [0,1)$ が存在するとき、このような $T$ を縮小写像という。

このとき、$T(x) = x$ を満たす点 $\tilde{x}$ が $X$ に唯一存在する。（不動点という）

## 平均値形式

**定義**　写像 $f: D \rightarrow \mathbb{R}^{n}$ が区間 $X\subset D$ ($D$ は $f$  の定義域)において、1階連続微分可能とする。
このとき、$x,~\tilde{x} \in X$に対して、

$$
f(x) \in f(\tilde{x})+Df_{[]}(X)(x-\tilde{x})
$$

が成立する。（$Df_{[]}(x)$ は、写像 $f$ のヤコビ行列 $J(x)$ の区間 $X$ における区間拡張）上記の右辺を写像 $f$ の**平均値形式**と呼ぶ。

## 自動微分を利用したヤコビ行列の計算

Kwawczyk法を使う際には、区間拡張 $Df_{[]}(X)$ を計算する必要がある。計算の方法として、最も標準的な実装方法は、自動微分を利用することである。

Juliaで自動微分を適用する場合には、ForwardDiffパッケージを利用する。使用例は以下の通りである。以下では $f(x,y) = x^2+y^2-1$, $g(x,y) = x^3 + y^4$ として

$$
    h(x,y) = \left(\begin{array}{c}f(x,y)\\ g(x,y)\end{array}\right)
$$

のヤコビ行列

$$
J(x) = \left(\begin{array}{cc}f_x(x,y) & f_y(x,y)\\ g_x(x,y) & g_y(x,y)\end{array}\right)= \left(\begin{array}{cc} 2x & 2y\\ 3x^2 & 4y^3\end{array}\right)
$$

の区間 $(x,y) = ([0.8,0.9], [-1,-0.5])$ における値を求める。

In [57]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff
X = [(0.8..0.9),(-1..(-0.5))]
f(x, y) = x^2 + y^2 - 1
g(x, y) = x^3 + y^4
h( (x, y) ) = [f(x, y); g(x, y)]
# ForwardDiff.jacobian(g, X::IntervalBox) = ForwardDiff.jacobian(g, X.v)
J = ForwardDiff.jacobian(h,X)

2×2 Array{Interval{Float64},2}:
 [1.59999, 1.80001]  [-2, -1]
 [1.91999, 2.43001]   [-4, -0.5]

$h(x,y)$ の区間　$X = ([0.8,0.9], [-1,-0.5])$ における区間拡張 $h_{[]}(X)$ は

In [53]:
h(X)

2-element Array{Interval{Float64},1}:
 [-0.110001, 0.810001]
  [0.574499, 1.72901]

## Krawczyk法の実装

ここから、Juliaを使ったKrawczyk法の実装を行う。実装例は以下の通りである。

In [58]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff
#区間Xを定義
X = [(0.6..0.8),(0.6..0.8)]
#計算対象となる方程式を定義
f(x, y) = x^2 + y^2 - 1
g(x, y) = x - y
F( (x, y) ) = [f(x, y); g(x, y)]
# ForwardDiff.jacobian(g, X::IntervalBox) = ForwardDiff.jacobian(g, X.v)
#J=f'(I)
#区間Xにおけるヤコビ行列を計算
iDF = ForwardDiff.jacobian(F,X) # 区間演算必要

2×2 Array{Interval{Float64},2}:
     [1.19999, 1.60001]        [1.19999, 1.60001]
 [1, 1]                  [-1, -1]

In [59]:
#mid(X)を定義
c = mid.(X)
ic = map(Interval,c)
#Rを計算するためのf'(c)を求める
DF = ForwardDiff.jacobian(F,c) # 区間演算なしでOK

2×2 Array{Float64,2}:
 1.4   1.4
 1.0  -1.0

In [60]:
#区間Xにおけるヤコビ行列Jの逆行列Rを定義する
R = inv(DF) # 区間演算なし

2×2 Array{Float64,2}:
 0.357143   0.5
 0.357143  -0.5

In [61]:
#単位行列を生成
# Matrix{Float64}(I,2,2)
M = Matrix{Float64}(I,2,2) - R*iDF # 区間演算必要

2×2 Array{Interval{Float64},2}:
 [-0.0714286, 0.0714286]  [-0.0714286, 0.0714286]
 [-0.0714286, 0.0714286]  [-0.0714286, 0.0714286]

In [59]:
R*F(ic) # 区間演算必要

2-element Array{Interval{Float64},1}:
 [-0.00714286, -0.00714285]
 [-0.00714286, -0.00714285]

In [62]:
#Krawczyk写像を計算
K = c - R*F(ic) + M*(X - c)　# 区間演算必要

2-element Array{Interval{Float64},1}:
 [0.692857, 0.721429]
 [0.692857, 0.721429]

In [63]:
#収束判定
K .⊂ X

2-element BitArray{1}:
 1
 1

In [76]:
function krawczyk(F,X)
    iDF = ForwardDiff.jacobian(F,X);
    c = mid.(X); ic = map(Interval,c);
    DF = ForwardDiff.jacobian(F,c);
    R = inv(DF);
    M = Matrix{Float64}(I,2,2) - R*iDF;
    return c - R*F(ic) + M*(X - c)
end

krawczyk (generic function with 2 methods)

In [65]:
 K = krawczyk(F,X);
 tol = 5e-10;
 while maximum(radius.(K)) >= tol
     K = krawczyk(F,K)
 end
 @show K
 radius.(K)


K = Interval{Float64}[[0.707106, 0.707107], [0.707106, 0.707107]]


2-element Array{Float64,1}:
 1.9761969838327786e-14
 1.9761969838327786e-14

In [66]:
err = 0.7;
X = [0.7 ± err , 0.7 ± err ];
c = mid.(X);
K = krawczyk(F,X);
K .⊂ X

2-element BitArray{1}:
 0
 0

**上記のように**、候補区間 $X$ の選び方次第では、$K(X)\subset \mathrm{int}(X)$が成立しない場合もある。その時は、柏木の方法によって、候補区間$X$を変更する。以下に、その手順を示す。

いま $c\in \mathbb{F}^m$ を非線形方程式の数値計算で得られた近似解とする。
ここで、$r=|Rf(c)|\in \mathbb{F}^m$ をベクトルとして考え、候補区間 $X$ を

$$
X=\left(\begin{array}{c}
{\left[c_{1}-u_{1}, c_{1}+u_{1}\right]} \\
{\left[c_{2}-u_{2}, c_{2}+u_{2}\right]} \\
\vdots \\
{\left[c_{m}-u_{m}, c_{m}+u_{m}\right]}
\end{array}\right),\quad u_{i}=r_{i}+\frac{1}{n} \Sigma_{k} r_{k}
$$

とする。そうすることで、$K(X)\subset \mathrm{int}(X)$ がより成立するようになる。以下に実装方法を示す。

In [67]:
#rを計算
r = abs.(R*F(c))

2-element Array{Float64,1}:
 0.00714285714285707
 0.00714285714285707

In [68]:
#uを計算
u = r .+ (sum(r)/length(r))

2-element Array{Float64,1}:
 0.01428571428571414
 0.01428571428571414

In [69]:
#候補区間Xを新たに定める
X_new = c .± u;
X = X_new;
K = krawczyk(F,X);
K .⊂ X

2-element BitArray{1}:
 1
 1

今までの結果を用いて、最終的なKrawczyk法のアルゴリズムを構築する。

In [83]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff

#候補区間Xを定義
X = [(0.6..0.7),(0.6..0.8)]

#計算対象となる方程式を定義
f(x, y) = x^2 + y^2 - 1
g(x, y) = x^2 - y^4
F( (x, y) ) = [f(x, y); g(x, y)]


In [87]:
function newton(F,x0)
    tol = 5e-10; count = 0;
    x = x0;
    Fx = F(x);
    while maximum(abs.(Fx)) ≥ tol && count ≤ 20
        DF = ForwardDiff.jacobian(F,x);
        x -= DF\Fx;
        Fx = F(x);
        count += 1;
    end
    return x
end

function krawczyk(F,X,R)
    c = mid.(X); ic = map(Interval,c);
    iDF = ForwardDiff.jacobian(F,X);
    M = Matrix{Float64}(I,2,2) - R*iDF;
    return c - R*F(ic) + M*(X - c)
end

function verifynlss_krawczyk(F, c)
    DF = ForwardDiff.jacobian(F,c);
    R = inv(DF);
    r = abs.(R*F(c));
    u = r .+ (sum(r)/length(r));
    X = c .± u;
    K = krawczyk(F,X,R);
    if all(K .⊂ X)
        tol = 5e-10;
        while maximum(radius.(K)) >= tol
            K = krawczyk(F,K,R)
        end
        success = 1
        return success, K
    end
    println("Oh my way, verification is failed...return a improved approximate solution") # cをNewton法で改善しても良い。
    success = 0
    return success, newton(F,c)
end


success, X = verifynlss_krawczyk(F,[0.6,0.7]);
if success == 0
    success, X = verifynlss_krawczyk(F,X);
end

Oh my way, verification is failed...return a improved approximate solution


(1, Interval{Float64}[[0.618033, 0.618034], [0.786151, 0.786152]])

In [55]:
K

2-element Array{Interval{Float64},1}:
 [0.577318, 0.65652]
 [0.757529, 0.837204]

## 区間ニュートン法

次に、もう1つの手法である区間ニュートン法について説明する。区間ニュートン法は、G.Alefeldによって提案された手法である。主張は以下の通りである。

与えられた区間ベクトル$X\in \mathbb{IR}^n$に対して、$f:X\rightarrow \mathbb{R}^n$が1階連続微分可能な関数とする。$M\in f'_{[]}(X)$を満たす任意の行列$M$が正則であると仮定し、ある$x_0\in X$に対して、集合$N(x_0,X)$を

$$
N(x_0,X) = \{x_0 - M^{-1}f(x_0)|M\in f'_{[]}(X)\}
$$

と定義する。この時、$N(x_0,X)\subset X$が成立するならば、非線形方程式の真の解$x^*$が区間ベクトル$X$内に一意存在する。また、$N(x_0,X)\cap X=$ならば、非線形方程式の解は$X$内に存在しない。さらに、$x^*\in N(x_0,X)$である。

### Krawczyk法との違い

Krawczyk法と比較すると、解の存在検証部分は同一のプログラムで動作する。
大きく違う部分は、区間連立1次方程式を解く必要がある点である。Krawczykは、近似解$c$におけるヤコビ行列$J(c)$の逆近似行列$R$だけを利用する手法である。区間連立1次方程式は要素数が大きくなると計算速度が遅くなり、さらに精度が悪くなるという問題点がある。

したがって、区間ニュートン法とKrawczyk法は問題によって使い分けるのがよい。

### 区間ニュートン法の実装


以下では $f(x,y) = x^2+y^2-1$, $g(x,y) = x - y$ として問題を解く


In [23]:
using LinearAlgebra,IntervalArithmetic, ForwardDiff
#区間Xを定義
X = [(0.6..0.8),(0.6..0.8)]
c = mid.(X)
ic = map(Interval,c)
#計算対象となる方程式を定義
f(x, y) = x^2 + y^2 - 1
g(x, y) = x - y
F( (x, y) ) = [f(x, y); g(x, y)]

#区間Xにおけるヤコビ行列を計算
A = ForwardDiff.jacobian(F,X) # 区間演算必要
M = inv(A)
b = F(ic)

2-element Array{Interval{Float64},1}:
     [-0.0200001, -0.02]
 [-0, 0]

In [24]:
include("IntervalLinearAlgebra.jl"); # int_mulを使用する
function verifylss_iAib(iA,ib) # verify the solution element-wisely
    b = mid.(real(ib)) + mid.(imag(ib))*im;
    x̄ = iA\ib;
    n = length(x̄);
    R = inv(iA);
    #########
    #C_mid, C_rad = mm_ufp(R,A);
    G = Matrix{Float64}(I, n, n) - int_mul(R,iA);
    α = opnorm(G,Inf)# Interval arithmetic
    #########
    if α < 1
        x̄ = map(Interval,x̄);
        r = A*x̄ - ib; # Interval arithmetic
        Rr = R*r;
        err = abs.(Rr) + supremum(norm(Rr,Inf))/(1-α)*(abs.(G)*ones(n)); # Interval arithmetic
    else
        println("Oh my way, verification is failed...")
        err = nan;
    end
    return x = (real(x̄) .± supremum.(err)) + im*(imag(x̄) .± supremum.(err))
end

verifylss_iAib (generic function with 1 method)

In [25]:
verifylss_iAib(M,b)

MethodError: MethodError: no method matching atomic(::Type{Interval{Interval{Float64}}}, ::Interval{Float64})
Closest candidates are:
  atomic(!Matched::Type{Interval{T}}, ::Interval) where T<:AbstractFloat at /Users/otanishunsuke/.julia/packages/IntervalArithmetic/sRFlx/src/intervals/conversion.jl:123
  atomic(!Matched::Type{Interval{T}}, ::S) where {T<:AbstractFloat, S<:Real} at /Users/otanishunsuke/.julia/packages/IntervalArithmetic/sRFlx/src/intervals/conversion.jl:96

### 参考文献

1. 大石進一編著, 精度保証付き数値計算の基礎, コロナ社, 2018.<br>
(精度保証付き数値計算の教科書. 浮動小数点数および区間演算に詳しい. 今回は6章を参考にした)
1. 非線形方程式の解の精度保証 (+ 自動微分)<br>
http://www.kashi.info.waseda.ac.jp/~kashi/lec2020/nac/krawczyk.pdf